In [ ]:
# import modules
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from mayavi import mlab
import numpy as np
import os

In [ ]:
# extract frames from videos
def video_to_frames(video_dir, imgs_dir):
    cap = cv2.VideoCapture(video_dir)
    
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 500)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 500)
    
    video_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) - 1
    count = 0
    
    while cap.isOpened():
        _, frame = cap.read()
        cv2.imwrite(imgs_dir + "/%#05d.jpg" % (count), frame)
        count += 1
        
        if count == video_length:
            cap.release()
            break

In [ ]:
# convert RGB to normalized intensity
def rgb_to_intensity(img_path):
    img_rgb = mpimg.imread(img_path)
    x, y, _ = img_rgb.shape

    img_intensity = np.zeros((x, y))
    for i in range(x):
        for j in range(y):
            r, g, b = img_rgb[i][j]
            img_intensity[i][j] = (0xFFFF * r + 0xFF * g + b) / (0xFFFF + 0xFF + 1) / 0xFF
    return img_intensity

In [ ]:
# get video frames
video_to_frames('clip.mp4', './test')

# remove bad frames. Mostly initial and final frames
for img_path in img_paths:
    try: 
        mpimg.imread(img_path)
    except OSError:
        os.remove(img_path)
    else:
        pass

In [ ]:
# asemble images into a data cube: each image is a 2D matrix
img_folder = './test'
img_paths = [os.path.join(img_folder, file_name).replace("\\","/") for file_name in os.listdir(img_folder)]

# get the cube
(x, y, _), z = mpimg.imread(img_paths[0]).shape, len(img_paths)
data_cube = np.zeros((x, y, z))
for i in range(z):
    data_cube[:, :, i] = rgb_to_intensity(img_paths[i])

# save cube for furture use
np.save('datacube.npy', data_cube)

In [ ]:
# re-load the cube
cube = np.load("./datacube.npy")

# plot a random image to do a quick check
plt.imshow(cube[:, :, 65], cmap = 'plasma');


# plot spectra at regions where one of the right device has defect
z = cube.shape[2]
bias = np.linspace(-2, 2, z )

fig, (ax1, ax2) = plt.subplots(2)
fig.subplots_adjust(hspace = 0.65)
fig.set_size_inches(12,3)

fig.text(0.08, 0.5, 'Normalized Signal (a.u.)', va='center', rotation='vertical')
ax2.set_xlabel('Sample Bias (V)')

ax1.set_title('Spectra from Normal Device')
ax2.set_title('Spectra from Defected Device')

for i in range(83, 88):
    for j in range (27, 33):
        ax1.plot(bias, cube[i,j,:], alpha = 0.2, color = 'r');

for i in range(85, 90):
    for j in range (102, 108):
        ax2.plot(bias, cube[i,j,:], alpha = 0.2, color = 'r');

fig.savefig(fname = 'Compared Spectra.png', dpi=2**8)

In [ ]:
# use Mayavi for high-dimensional visualization
cube = np.load("./datacube.npy")
n, m, l = cube.shape

X, Y, Z = np.mgrid[0.:n, 0.:m, 0.:l]
X *= 1. / n
Y *= -1. / m  # flip Y
Y += 2  # Mayavi seems to have an offset
Z *= 1. / l

scalar_field = mlab.pipeline.scalar_field(X, Y, Z, cube)

scalar_cut_plane_1 = mlab.pipeline.scalar_cut_plane(
    scalar_field, plane_orientation="z_axes")
scalar_cut_plane_1.implicit_plane.widget.origin = [.5, .5, .5]

del X, Y, Z, cube